In [33]:
import time
import psutil
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.metrics import (
    accuracy_score, f1_score, roc_auc_score, classification_report,
    mean_squared_error, mean_absolute_error, r2_score
)

from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier, XGBRegressor
import joblib
from ucimlrepo import fetch_ucirepo


In [44]:

'''
[Covertype](https://archive.ics.uci.edu/dataset/31/covertype)

[Adult](https://archive.ics.uci.edu/dataset/2/adult)

[Bank Marketing](https://archive.ics.uci.edu/dataset/222/bank+marketing)

[Poker Hand](https://archive.ics.uci.edu/dataset/158/poker+hand)

[HIGGS](https://archive.ics.uci.edu/dataset/280/higgs)

[Wine Quality](https://archive.ics.uci.edu/dataset/186/wine+quality)
'''

seed = 999
dataset_name = "Covertype" 
dataset_id = 31
task_type = "classification"  # or "regression"
scale = 0.5

#save_dir = os.path.join(r"D:\DSS5104\final\xgboost\scale", dataset_name)
save_dir = os.path.join(r"D:\DSS5104\final\xgboost\scale", f"{dataset_name}_scale{scale}")
os.makedirs(save_dir, exist_ok=True)


In [24]:
from sklearn.datasets import fetch_california_housing
seed = 888

def load_california_housing():
    data = fetch_california_housing()
    X = data.data 
    y = data.target 
    
    feature_names = data.feature_names
    X = pd.DataFrame(X, columns=feature_names)

    return X, y
dataset_name = "California Housing"  
task_type = "regression"

save_dir = os.path.join(r"D:\DSS5104\final\xgboost\888", dataset_name)
os.makedirs(save_dir, exist_ok=True)


In [4]:
def load_data():
    dataset = fetch_ucirepo(id=dataset_id)
    X, y = dataset.data.features, dataset.data.targets.squeeze()

    # 💡 清洗标签数据
    y = y.astype(str).str.strip().str.replace('.', '', regex=False)

    if (X == "?").any().any():
        print("🔍 检测到 '?' 缺失值，正在清洗并填充...")
        X = X.replace("?", np.nan)

        original_shape = X.shape[0]
        df = pd.concat([X, y], axis=1)
        df = df.dropna()
        removed = original_shape - df.shape[0]
        print(f"🧹 检测到 '?'，已删除包含缺失值的样本，共删除 {removed} 条记录")

        X = df.drop(columns=[y.name])
        y = df[y.name]

    return X, y


In [35]:
#sample 0.1, 0.5
def load_data():
    dataset = fetch_ucirepo(id=dataset_id)
    X, y = dataset.data.features, dataset.data.targets.squeeze()
    
    # 合并并采样
    data = pd.concat([X, y], axis=1)
    data = data.sample(frac=scale, random_state=seed)  # 使用全局变量
    
    # 重新拆分
    X = data.drop(columns=[y.name])
    y = data[y.name]
    
    # 原有清洗逻辑
    y = y.astype(str).str.strip().str.replace('.', '', regex=False)
    if (X == "?").any().any():
        X = X.replace("?", np.nan)
        df = pd.concat([X, y], axis=1).dropna()
        X, y = df.drop(columns=[y.name]), df[y.name]
    
    return X, y

In [36]:
def split_features(X):
    cat_cols = X.select_dtypes(include=["object"]).columns.tolist()
    num_cols = X.select_dtypes(include=["float64", "int64"]).columns.tolist()
    return num_cols, cat_cols

def build_preprocessor(num_cols, cat_cols):
    numeric_transformer = StandardScaler()
    categorical_transformer = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
    return ColumnTransformer([
        ('num', numeric_transformer, num_cols),
        ('cat', categorical_transformer, cat_cols)
    ])


In [45]:
#X, y = load_california_housing()
X, y = load_data()
num_cols, cat_cols = split_features(X)
preprocessor = build_preprocessor(num_cols, cat_cols)

if task_type == "classification":

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2,
        stratify=y if task_type == "classification" else None,
        random_state=seed
    )

else:
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2,
        random_state=seed
    )


print(f"✅ 数据集大小: 训练={len(X_train)}, 测试={len(X_test)}")

✅ 数据集大小: 训练=232404, 测试=58102


In [46]:
model = XGBClassifier(n_jobs=-1, random_state=seed, use_label_encoder=False, eval_metric='logloss') \
    if task_type == "classification" \
    else XGBRegressor(n_jobs=-1, random_state=seed)

pipeline = Pipeline([
    ("pre", preprocessor),
    ("xgb", model)
])


In [49]:
# 🚀 模型训练
start_time = time.time()
cpu_before = psutil.cpu_percent(interval=1)

# Encode if classification
if task_type == "classification":
    label_encoder = LabelEncoder()
    y_train_encoded = label_encoder.fit_transform(y_train)
    pipeline.fit(X_train, y_train_encoded)
else:
    pipeline.fit(X_train, y_train)

train_time = time.time() - start_time
cpu_after = psutil.cpu_percent(interval=1)

print(f"\n⏱️ 模型训练完成：耗时 {train_time:.2f} 秒")
print(f"🧠 CPU 使用率：训练前 = {cpu_before}%，训练后 = {cpu_after}%")
print(f"💾 CPU占据量：{cpu_after - cpu_before}")

d:\python\Lib\site-packages\xgboost\training.py:183: UserWarning: [00:18:03] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



⏱️ 模型训练完成：耗时 15.14 秒
🧠 CPU 使用率：训练前 = 15.7%，训练后 = 18.9%
💾 CPU占据量：3.1999999999999993


In [50]:
# 🎯 预测与评估
y_test_pred = pipeline.predict(X_test)

results = {
    "train_time_seconds": train_time,
    "cpu_before_percent": cpu_before,
    "cpu_after_percent": cpu_after
}

if task_type == "classification":
    y_test_encoded = label_encoder.transform(y_test)

    test_acc = accuracy_score(y_test_encoded, y_test_pred)
    test_f1 = f1_score(y_test_encoded, y_test_pred, average="macro")

    results.update({
        "test_accuracy": test_acc,
        "test_f1": test_f1,
    })

    print(f"\n📊 分类评估指标：")
    print(f"✅ 测试集 Accuracy: {test_acc:.4f}")
    print(f"✅ 测试集 F1 Score: {test_f1:.4f}")

    try:
        y_test_proba = pipeline.predict_proba(X_test)
        test_auc = roc_auc_score(y_test_encoded, y_test_proba[:, 1])
        results["test_auc"] = test_auc
        print(f"✅ 测试集 AUC: {test_auc:.4f}")
    except Exception as e:
        test_auc = np.nan
        print("⚠️ 无法计算 AUC:", e)

else:
    test_rmse = mean_squared_error(y_test, y_test_pred) ** 0.5
    test_mae = mean_absolute_error(y_test, y_test_pred)
    test_r2 = r2_score(y_test, y_test_pred)

    results.update({
        "test_rmse": test_rmse,
        "test_mae": test_mae,
        "test_r2": test_r2,
    })

    print(f"\n📊 回归评估指标：")
    print(f"✅ 测试集 RMSE: {test_rmse:.4f}")
    print(f"✅ 测试集 MAE: {test_mae:.4f}")
    print(f"✅ 测试集 R²: {test_r2:.4f}")



📊 分类评估指标：
✅ 测试集 Accuracy: 0.8648
✅ 测试集 F1 Score: 0.8453
⚠️ 无法计算 AUC: multi_class must be in ('ovo', 'ovr')


In [51]:
joblib.dump(pipeline, os.path.join(save_dir, f"{dataset_name}_xgboost_model.pkl"))

# 按照指定顺序组织结果字典
if task_type == "classification":
    ordered_results = {
        "Accuracy": test_acc,
        "F1_score": test_f1,
        "AUC": test_auc,
        "time": train_time,
        "cpu_before(%)": cpu_before,
        "cpu_after(%)": cpu_after,
        "cpu_occupied(%)": cpu_after - cpu_before
    }
else:
    ordered_results = {
        "RMSE": test_rmse,
        "MAE": test_mae,
        "R2": test_r2,
        "time": train_time,
        "cpu_before(%)": cpu_before,
        "cpu_after(%)": cpu_after,
        "cpu_occupied(%)": cpu_after - cpu_before
    }

# 保存评估结果
result_df = pd.DataFrame([ordered_results])
result_path = os.path.join(save_dir, f"{dataset_name}_xgboost_results.xlsx")
result_df.to_excel(result_path, index=False)

# 📊 特征重要性图
xgb_model = pipeline.named_steps["xgb"]
feature_names = num_cols + cat_cols
importance = xgb_model.feature_importances_
top_idx = np.argsort(importance)[::-1][:len(feature_names)]

importance_df = pd.DataFrame({
    "Feature": np.array(feature_names)[top_idx],
    "Importance": importance[top_idx]
})

with pd.ExcelWriter(os.path.join(save_dir, f"{dataset_name}_xgboost_results.xlsx"), mode="a", engine="openpyxl") as writer:
    importance_df.to_excel(writer, sheet_name="Feature Importance", index=False)

top_n = 10
top_idx = np.argsort(importance)[::-1][:top_n]  

plt.figure(figsize=(10, 4))
plt.bar(range(len(top_idx)), importance[top_idx])
plt.xticks(range(len(top_idx)), np.array(feature_names)[top_idx], rotation=45)
plt.title("Top 10 Feature Importances")
plt.tight_layout()
plt.savefig(os.path.join(save_dir, f"{dataset_name}_feature_importance.png"))
plt.close()

print("✅ 所有模型和评估结果已保存完毕 ✅")

✅ 所有模型和评估结果已保存完毕 ✅
